# Part 1: Theoretical Overview

## Project structure

N-body problem

NS equations
 - Lagrangian description here (EUlerian in Fluid Dynamics course)
 - SPH solution strategy
Shock test

Colliding planets with gravity


## The Navier-Stokes Equations

The equation of fluid dynamics (in Lagrangian form) are widely used for fluid dynamics problems ($\sigma$ is the total stress tensor) to model fluid motion

\begin{align*}
&\frac{D\boldsymbol{x}^\alpha}{Dt} = \boldsymbol{v}^\alpha\\
&\frac{D\rho}{Dt} = -\rho\frac{\partial\boldsymbol{v}^\beta}{\partial\boldsymbol{x}^\beta}\\
&\frac{D\boldsymbol{v}^\alpha}{Dt} = \frac{1}{\rho}\frac{\partial\sigma^{\alpha\beta}}{\boldsymbol{x}^\beta} + F_{Gravity}\\
&\frac{De}{Dt} = \frac{\sigma^{\alpha\beta}}{\rho}\frac{\partial\boldsymbol{v}^\alpha}{\partial\boldsymbol{x}^\beta} + E_{Heat}
\end{align*}


### Solution strategy of meshfree particle methods like SPH:

Approximate the functions - derivatives and integrals - using the information from neighbouring particles in an area of influence withing the support domain.

E.g. the velocity $v$ of a particle at a position $\vec{x}$ is

\begin{equation}
v(\vec{x}) = \sum_{i=1}^N v(\vec{x}_i)W(\vec{x}_i),
\end{equation}



where N is the number of particles, $v_i = v(x_i)$ is the velocity at particle $i$, $W_i$ is a smoothing function (kernel) at the $i$:th particle. 

Velocity is thus a weighted sum over neighbouring particles.

Every particle has a smoothing function affecting all other particles.

Disadvantages of smoothed particle hudrodynamics (SPH):

- We don't always want the smoothing effect.
- Sharp effects are lost




## The SPH formulation

### introduction

In this text we introduce:

- the basic concepts of SPH
- The concepts of the support and influence domains
- how to derive SPH formulations

The formulations of SPH is usually divided into two main steps
- The kernel approximation (or integral representation)
- The discretised particle approximation


### Basic Concepts

In the SPH method we use the following key ideas:

1. The problem domain is represented by a set of arbitrarily distributed particles and no connectivity of the particles is needed (i.e. meshfree).

2. We use an integral representation for the field functions and introduce a kernel approximations. It provides stability and has a smoothing effect

3. The kernel approximation is then further approximated using particles. This is known asa the particle approximation in SPH
    - Done by replacing the integral representation of the functions and its derivatives with summation over all corresponding values at the neighbouring particles in a local domain called the support domain (compact support)

    ```math 
    W(|x_i-x_j|,h)
    ```

    - Results in banded sparse matrices which can be solved efficiently

4. The particle approx is performed at each time step, so particles depend on the current local distrib of particles (adaptive). Therefore, SPH naturally handles problems with large deformation.

5. The support domain must be sufficiently large and particles can be assigned mass and become physical material particles.

6. The particle approximations are performed on all field functions to produce a set of ODE's in discretised form with respecti to time only (Lagrangian). Such equations are conceptually simpler than the Eulerian equvalent. 

7. The ODE's are solved using an integration algorithm to achieve fast time stepping, and to obtain the time history of all the field variables for all the particles (dynamic).

So our dynamical problem is meshfree, adaptive, stable and Lagrangian.

The detailed formulation follows below.



### Mathematical overview of SPH formulations

#### Integral representation of a field function (1st main step):
The concept of the integral representatin (or kernel approximation) of a function $f(\boldsymbol{x})$ is to write, for some weightfunction:

\begin{equation*}
f(x) = \int_\Omega f(x^\prime)\delta(x-x^\prime)dx^\prime
\end{equation*},

where $f$ is a function of the position vector x. This is mean that we essentially consider the convolution of $f$ with the Dirac delta distribution. Recall that

\begin{align*}
\delta(\boldsymbol{x}-\boldsymbol{x}^\prime) = \begin{cases}\infty \qquad &\boldsymbol{x} = \boldsymbol{x}^\prime\\
0 \qquad &\boldsymbol{x} \neq \boldsymbol{x}^\prime,
\end{cases}
\end{align*}

and

\begin{equation*}
\int_{-\infty}^\infty \delta(x)dx = 1
\end{equation*}


This relation for $f(\boldsymbol{x})$, is exact as long as f(x) is defined and continuous on $\Omega$. 

We can now replace the Dirac delta by a smoothing function (kernel), $W(x-x^\prime,h)$, such that

\begin{equation*}
\boxed{f(x) \approx \langle f(x)\rangle \equiv \int_\Omega f(x^\prime)W(x-x^\prime,h)dx^\prime.} \quad \text{(kernel approximation)}
\end{equation*}

The kernel is chosen to be an *even* function and it needs to be normalized, i.e., we require

\begin{align*}
\int_\Omega W(x-x^\prime, h)dx^\prime = 1.
\end{align*}

Secondly, the Delta function should be recovered when the smoothing length, $h$, goes to zero,

\begin{equation*}
\lim_{h\rightarrow 0}W(x-x^\prime,h) = \delta(x-x^\prime).
\end{equation*}

The third condition is the compact condition

\begin{equation*}
W(x-x^\prime,h) = 0\:\text{when } |x-x^\prime| > \kappa h,
\end{equation*}

where $\kappa$ is a constant and defines the non-zero area of the smoothing function called the **support domain**. This condition localises the support domain of the smoothing function.

The kernel approximation is of $2^{nd}$ order accuracy if $W$ is an even function. We can show this by expanding $f(x^\prime)$ around $x$ using a Taylor series. From
\begin{equation*}
\langle f(x)\rangle = \int_\Omega f(x^\prime)W(x-x^\prime,h)dx^\prime,
\end{equation*}
we get
\begin{align*}
    \langle f(x)\rangle &= \int_\Omega\left[f(x) + f^\prime(x)(x^\prime - x) + \mathcal{O}((x^\prime - x)^2)\right]W(x-x^\prime,h)dx^\prime\\
    &= f(x)\int_\Omega W(x-x^\prime,h)dx^\prime\\
    &\quad +f^\prime(x)\int_\Omega(x^\prime - x)W(x-x^\prime,h)dx^\prime \\
    &\quad +\mathcal{O}(h^2)\mathcal{O}((x-x^\prime)^2)\\
    &= f(x) + \mathcal{O}(r²),
\end{align*}

where we have used that the term $\int_\Omega(x-x^\prime)Wdx^\prime$ is zero, since $W$ is an even function and $(x-x^\prime)$ is an odd function (a straight line). 

For this to be applicable to the terms, e.g. $v(\boldsymbol{x})$, in the NS equations, we are also interested in the divergence of functions in this kernel approximation. In other words, given the above integral representation of $f(x)$, we want to calculate the divergence of $f(x)$. We therefore evaluate

\begin{align*}
\nabla\cdot f(x) = \int_\Omega \nabla\cdot[f(x^\prime)W(x-x^\prime,h)]dx^\prime.
\end{align*}

We apply the product rule $\nabla\cdot[AB] = \nabla\cdot AB + A\nabla\cdot B$, so that

\begin{align*}
    \langle \nabla\cdot f(x)\rangle &= 
    \int_\Omega\nabla\cdot[f(x^\prime)W(x-x^\prime,h)]dx^\prime\\
    &\quad - \int_\Omega f(x^\prime)\cdot\nabla W(x-x^\prime,h)dx^\prime.
\end{align*}

The first integral in the RHS can be rewritten using the divergence theorem, while the second integral depends on the gradient of the weight function, $W$, which is analytically obtainable, i.e., we often choose $W$ to be some analytic expression, e.g., a Gaussian.

Carrying out the divergence theorem we see that
\begin{align*}
\langle \nabla\cdot f(x)\rangle &= \int_{\partial\Omega}f(x^\prime)W(x-x^\prime,h)\cdot\boldsymbol{n}\:dS \\
&\quad- \int_\Omega f(x^\prime)\cdot\nabla W(x-x^\prime,h)dx^\prime\\
&= - \int_\Omega f(x^\prime)\cdot\nabla W(x-x^\prime,h)dx^\prime.
\end{align*}
Since the kernel is compact, the surface integral is simply zero, and vanishes. *Note*: this assumes the support domain does not overlap the problem boudary as this would be *non-zero*.

Finally, then, we have that

\begin{equation*}
\boxed{\langle \nabla\cdot f(x)\rangle = -\int_\Omega f(x^\prime)\cdot\nabla W(x-x^\prime,h)dx^\prime.}
\end{equation*}

The spatial divergence of the field function can therefore be calculated from the field function and the derivatives of the kernel function $W$, rather than from the derivatives of the field function itself. 


#### Particle approximation (2nd main step):

In practical reality, objects are represented by a finite number of particles that carry mass and occupy individual spaces. This can be done by applying a particle approximation, where the integrals for $\langle f(x)\rangle$ and $\langle \nabla\cdot f(x)\rangle$ can be converted into forms with summation over all the particles in the support domain.

If the infinitesimal volume $dx^\prime$ at the location of particle $j$ is replaced by the finite volume of the particle $\Delta V_j$ that is related to the mass of particles $m_j$ by:

\begin{equation*}
    m_j = \Delta V_j\rho_j,
\end{equation*}
where density $\rho_j$ is the mass per unit volume of the particle $j(=1,2,...,N)$, given $N$ total particles.

The SPH representation for $f(x)$ can be written in the particle approximation as
\begin{align*}
    \langle f(x)\rangle &= \int_\Omega f(x^\prime)W(x-x^\prime,h)dx^\prime \\
    & \approx \sum_{j=1}^N f(x_j)W(x-x_j,h)\Delta V_j\\
    &= \sum_{j=1}^Nf(x_j)W(x-x_j,h)\frac{1}{\rho_j}(\rho_j\Delta V_j)\\
    &= \sum_{j=1}^N f(x_j)W(x-x_j,h)\frac{1}{\rho_j}m_j,
\end{align*}
or
\begin{equation*}
\boxed{\langle f(x)\rangle = \sum_{j=1}^N\frac{m_j}{\rho_j}f(x_j)W(x-x_j,h).}
\end{equation*}

Similarly, the particle approximation for the spatial derivative becomes

\begin{equation*}
\boxed{\langle\nabla\cdot f(x)\rangle = -\sum_{j=1}^N\frac{m_j}{\rho_j}f(x_j)\cdot\nabla W(x-x_j,h).}
\end{equation*}

The value of the function at particle $i$ is approximated by using the average of the function at all particles in the support domain weighted by the smoothing function (kernel).


## Concluding remarks

SPH os a particle based meshfree approach.

There is no connectivity between the particles and only an initial particle distribution is needed.

The SPH approximation consists of a kernel approx in the continuum domain and a particle approx in the support domain at the current time step.



Reference: Annual rev. of astronomy and astrophys 30:543-574. Monaghan (1992).

# Part 2: Practical implementation

## Smoothing functions in more detail

The smoothing function (or kernel) is important as it

- determines the shape for the function approximation
- defines the dimensions of the support domain
- determines the accuracy of both the kernel and particle approximations

Different smoothing functions appear in the literature. Examples are given in the Lecture slides. 

We will use the **piecewise cubic** function, defined by

\begin{align*}
    W(R,h) = a_d\begin{cases}
    \frac{2}{3} - R^2 + \frac{1}{2}R^3\quad & 0 \leq R < 1\\
    \frac{1}{6}(2-R)^3 \quad & 1\leq R \leq 2
    \end{cases}
\end{align*}




## Navier-Stokes equations

The NS equations are an explicit statement of the conservation of mass, momentum and energy. They generalise the conservation equations in the Lagrangian frame.

1. The continuity equation

\begin{equation*}
    \frac{D\rho}{Dt} = -\rho\frac{\partial\boldsymbol{v}^\beta}{\partial\boldsymbol{x}^\beta}.
\end{equation*}

2. The momentum equation add $+F^\alpha$ for external forces

\begin{equation*}
    \frac{D\boldsymbol{v}^\alpha}{Dt} = \frac{1}{\rho}\frac{\sigma^{\alpha\beta}}{\partial\boldsymbol{x}^\beta}
\end{equation*}

3. The energy equation

\begin{equation*}
\frac{De}{Dt} = \frac{\sigma^{\alpha\beta}}{\rho}\frac{\partial\boldsymbol{v}^\alpha}{\partial\boldsymbol{x}^\beta}, \qquad \frac{De}{Dt} = -\frac{p}{\rho}\frac{\partial\boldsymbol{v}^\beta}{\partial\boldsymbol{x}^\beta} + \frac{\mu}{2\rho}\epsilon^{\alpha\beta}\epsilon^{\alpha\beta},
\end{equation*}

note that to include the coordinate indices properly we do double summation over the $\epsilon$ terms. The second equation is numerically more stable.

$\sigma$ is the total stress tensor. It is made up of the isotropic pressure $p$ and the viscous stress $\tau$.

\begin{equation*}
\sigma^{\alpha\beta} = -p\delta^{\alpha\beta} + \tau^{\alpha\beta}.
\end{equation*}

The viscous stress should be proportional to the strain rate $\epsilon$ through the dynamic viscosity $\mu$:
\begin{equation*}
    \tau^{\alpha\beta} = \mu\epsilon^{\alpha\beta},
\end{equation*}
with
\begin{equation*}
    \epsilon^{\alpha\beta} = \frac{\partial\boldsymbol{v}^\beta}{\partial\boldsymbol{x}^\alpha} + \frac{\partial\boldsymbol{v}^\alpha}{\partial\boldsymbol{x}^\beta} - \frac{2}{3}(\nabla\cdot\boldsymbol{v})\delta^{\alpha\beta}.
\end{equation*}
We will neglect viscousity here, (for now).


## Particle approximation for NS equations

\begin{align*}
    \text{Summation Density}\\
    \text{Continuity Densty}
\end{align*}

## Summation density (use this)

This directly applies the SPH approximation to the density itself. For a given particle $i$ we have

\begin{equation*}
\rho_i = \sum_{j=1}^N m_jW_{ij}, \qquad \text{alt.}\qquad \rho_i = \frac{\sum_j m_jW_{ij}}{\sum_j\left(\frac{m_j}{\rho_j}\right)W_{ij}},
\end{equation*}
where we sum to the nummber of support domain particles, $N$, and $m_j$ is the mass of particle $j$.

$W_{ij}$ is the kernel function of particle $i$ evaluated at $j$:

\begin{equation*}
    W_{ij} = W(x_i-x_j, h) = W(|x_i-x_j|,h) = W(R_{ij},h),
\end{equation*}
where 
\begin{equation*}
    R_{ij} = \frac{r_{ij}}{h} = \frac{|x_i-x_j|}{h}
\end{equation*}
is the relative distance of $i$ and $j$, and $r_{ij}$ is the scalar distance. $W_{ij}$ has units of inverse volume.


## Continuity density

Here the density is obtained from the continuity equation, possible forms are

\begin{equation*}
    \frac{D\rho_i}{Dt} = \sum_{j=1}^N m_jv_{ij}^\beta\frac{\partial W_{ij}}{\partial x_i^\beta},
\end{equation*}
where $v_{ij}^\beta = (v_i^\beta - v_j^\beta)$.

## Artificial Viscosity

Special treatment is needed to model shock waves, to avoid the simulation developing unphysical oscillations. 

The artificial viscosity we willl use is given by:
\begin{align*}
    \Pi_l = \begin{cases}
    a_l\Delta x^2\rho(\nabla\cdot\boldsymbol{v})^2 \quad &\nabla\cdot\boldsymbol{v} < 0\\
    0 \quad &\nabla\cdot\boldsymbol{v} \geq 0
    \end{cases}
\end{align*}

## Reducing the NS equation to first order DEs

(note: we want to put all particles into state-vectors, or matrices + flatten using python).

\begin{equation*}
    \boldsymbol{S} = \left[\boldsymbol{s_1},...,\boldsymbol{s}_N\right],
\end{equation*}
where $\boldsymbol{s}_i = [\boldsymbol{x}_i,\boldsymbol{v}_i,\rho_i,e_i]$ is a (single-particle) vector containing the cartesian position and velocity vectors. 


Evolution of the system takes the form
\begin{equation*}
    \frac{d\boldsymbol{S}}{dt} = g(\boldsymbol{S}).
\end{equation*}